Establish project and weights directories

In [ ]:
import sys
import os

# Go up two levels from notebook (Training/GRU) to project root
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_root)

print("Project root added to sys.path:", project_root)

# Ensure the model save directory exists
model_save_path = os.path.join('.')
os.makedirs(model_save_path, exist_ok=True)  # Creates directory if it doesn't exist

Load and preprocess data & Train the Model with Optuna

In [ ]:
# Load and preprocess data
from Training.Helper.dataPreprocessing import TRAIN_DATA_PATH_1990S, TRAIN_DATA_SPLIT, VAL_DATA_SPLIT, create_sequences, sklearn_fit_transform, add_dimension, prepare_dataloader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

# Read data into DataFrame, and identify target data (no exogenous data used)
df = pd.read_csv(TRAIN_DATA_PATH_1990S)
df["observation_date"] = pd.to_datetime(df["observation_date"], format="%m/%Y")
df = df.sort_values(by="observation_date").reset_index(drop=True)
target_col = "fred_PCEPI"

data = df[[target_col]].values.astype(np.float32)

Xy_train, Xy_val = train_test_split(data, train_size=TRAIN_DATA_SPLIT, test_size=VAL_DATA_SPLIT, shuffle=False)

#MinMax scale x and y
[Xy_train, Xy_val], y_scaler = sklearn_fit_transform(Xy_train, Xy_val, MinMaxScaler())
Xy_train, Xy_val = Xy_train.values, Xy_val.values  #get back out of DataFrame form

# Make sequences for model
sequence_length = 12
X_train_seq, y_train_seq = create_sequences(Xy_train, Xy_train, seq_len=sequence_length)
X_val_seq, y_val_seq = create_sequences(Xy_val, Xy_train, seq_len=sequence_length)

# Appropriately resize training and validation input sequences
[X_train_seq, X_val_seq] = [add_dimension(nparrays) for nparrays in [X_train_seq, X_val_seq]]

In [ ]:
# Get dataloaders
batch_size = 32
train_loader = prepare_dataloader(X_train_seq, y_train_seq, batch_size=batch_size)
val_loader = prepare_dataloader(X_val_seq, y_val_seq, batch_size=batch_size)

In [ ]:
import torch
from Models.GRU import GRUModel   # Import the GRU model
# Import modularized functions
from Training.Helper.PyTorchModular import optuna_tune_and_train

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Run Optuna & Train Model
model, metadata, study = optuna_tune_and_train(
    model_class=GRUModel, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    device=device,
    model_search_space={"hidden_size": (int, (32, 256)), "num_layers": (int, (1, 4))},
    optim_search_space={"lr": (float, (1e-5, 1e-1))},
    max_epochs=50,
    model_save_path=model_save_path,
    model_name="GRU_inflation",
    use_best_hyperparams=False,  # Set to False to force Optuna tuning every time
    n_trials=20,
    return_study=True,
    verbose=True
)

Adjustments for 48-Month Forecast & Evaluation

In [ ]:
# --- Make 48-month future predictions using last known sequence ---
import numpy as np
from sklearn.metrics import mean_squared_error
import math
import torch

# Prepare the starting sequence (last sequence in test set)
last_sequence = X_val_seq[-1].reshape(1, 12, 1)  # Shape: (1, 12, 1)
preds = []

model.eval()
with torch.no_grad():
    current_seq = torch.tensor(last_sequence, dtype=torch.float32).to(device)

    for _ in range(48):  # Predict next 48 steps
        next_step = model(current_seq)          # Shape: (1, 1)
        next_value = next_step.item()
        preds.append(next_value)

        # Shift window left and append prediction
        current_input = current_seq.cpu().numpy().reshape(12, 1)  # Always (12, 1)
        next_input = np.append(current_input[1:], [[next_value]], axis=0)  # Shape: (12, 1)
        current_seq = torch.tensor(next_input.reshape(1, 12, 1), dtype=torch.float32).to(device)

# Inverse transform if scaling was applied
if y_scaler is not None:
    preds = y_scaler.inverse_transform(np.array(preds).reshape(-1, 1)).flatten()

# Save predictions
output_path = os.path.join(project_root, "Predictions", "GRU.npy")
np.save(output_path, np.array(preds))
print(f" Saved 48-month forecast to: {output_path}")


# Calculate RMSE
mse = mean_squared_error(y_test_seq, y_pred)
rmse = math.sqrt(mse)
print(f"GRU Test MSE: {mse}")
print(f"GRU Test RMSE: {rmse}")


Make Predictions on Validation Set

*used in standardisation*

In [ ]:
from Evaluation.Helper.evaluation_helpers import evaluate_model

observation_dates = df['observation_date'][int(len(df)*TRAIN_DATA_SPLIT):]

# Evaluate Model
df_comparison, rmse = evaluate_model(
     model=model, 
     val_loader=val_loader, 
     y_scaler=y_scaler, 
     observation_dates=observation_dates, 
     device=device,
     verbose=True
)

In [ ]:
# Uncomment these to clean all weight files just produced
#from Training.Helper.weightFileCleaner import cleanWeightFiles
#cleanWeightFiles('GRU', earlyStopped=False)